In [9]:
from datasets import load_dataset

# dataset1 = load_dataset("jiandong/crimson-attck-vectors")
# dataset = load_dataset("Xcvddax/ATT")
# dataset3 = load_dataset("jiandong/crimson-attck-bge-vectors")

dataset = load_dataset("jiandong/crimson-attck-vectors",split='train')
# dataset2 = load_dataset("dear-daddy/attck",split='train')
 

In [13]:
# print(dataset[0]['embedding'])
# embeddings = dataset[:]['embedding']
# train_data and dataset2

[0.015339930541813374, 0.03623397648334503, 0.03089144639670849, 0.04181453585624695, -0.011432215571403503, -0.04504121094942093, -0.029331006109714508, 0.004730914253741503, -0.012662054970860481, 0.043084047734737396, 0.03171133995056152, -0.008708056062459946, 0.025667935609817505, -0.044194869697093964, 0.0002975417592097074, -0.0013488560216501355, -0.00781542994081974, 0.0005244173808023334, 0.01848726160824299, -0.025998538359999657, 0.04514700546860695, 0.021383333951234818, -0.05585850775241852, 0.004846624564379454, -0.024504218250513077, 0.019320378080010414, 0.020682457834482193, 0.00929652713239193, 0.06516826152801514, 0.010605710558593273, -0.019360050559043884, -0.04789761081337929, 0.0313410647213459, -0.05956124886870384, -0.006010343786329031, -0.03268992155790329, 0.030864998698234558, -0.01717807725071907, -0.02027251198887825, -0.03247833624482155, 0.03930196166038513, -0.036577802151441574, 0.03155265003442764, -0.03562566637992859, -0.023948805406689644, -0.021

In [ ]:
# language_dataset = load_dataset("fengtc/GuanacoDataset",split='train')
# print(language_dataset)

In [7]:
# print(language_dataset)
# print(dataset)
# print(dataset2)
# print(type(dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [14]:
from dotenv import load_dotenv
load_dotenv()

from  langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(dataset)
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

TypeError: load_local() missing 1 required positional argument: 'folder_path'

In [2]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
  output_texts = []
  for i in range(len(docs)):
      text = f"""id: {docs['id'][i]}\n ,attck_id: {docs['attck_id'][i]} , description: {docs['description'][i]} ,
        kill_chain_phases: {docs['kill_chain_phases'][i]} , domains: {docs['domains'][i]} , tactic_type: {docs['tactic_type'][i]}
        """
      output_texts.append(text)
  return output_texts


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)